# ECS 入门
在开始进行量化研究之前，我们先介绍如何构建一个示例程序，并且在 ECS 中运行。

### 定义参数

首先，我们给这个示例程序定义一些环境变量：

In [1]:
import boto3

aws_account_id = boto3.client('sts').get_caller_identity().get('Account')
repository_name = 'ecs-demo-php-simple-app'
aws_region = 'us-east-1'

### 创建镜像仓库和容器镜像

接下来，我们使用 SDK 为示例程序创建镜像仓库：

In [2]:
ecs = boto3.client('ecs', region_name=aws_region)
ecr = boto3.client('ecr', region_name=aws_region)

In [3]:
ecr.create_repository(repositoryName=repository_name)

{'repository': {'repositoryArn': 'arn:aws:ecr:us-east-1:364198545638:repository/ecs-demo-php-simple-app',
  'registryId': '364198545638',
  'repositoryName': 'ecs-demo-php-simple-app',
  'repositoryUri': '364198545638.dkr.ecr.us-east-1.amazonaws.com/ecs-demo-php-simple-app',
  'createdAt': datetime.datetime(2021, 12, 9, 7, 30, 35, tzinfo=tzlocal()),
  'imageTagMutability': 'MUTABLE',
  'imageScanningConfiguration': {'scanOnPush': False},
  'encryptionConfiguration': {'encryptionType': 'AES256'}},
 'ResponseMetadata': {'RequestId': 'bac11b3d-0441-4c00-81d9-fe2ca2fda04c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'bac11b3d-0441-4c00-81d9-fe2ca2fda04c',
   'date': 'Thu, 09 Dec 2021 07:30:35 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '423'},
  'RetryAttempts': 0}}

创建好镜像仓库之后，运行以下代码下载示例程序：

In [4]:
!git clone https://github.com/aws-samples/ecs-demo-php-simple-app

Cloning into 'ecs-demo-php-simple-app'...
remote: Enumerating objects: 79, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 79 (delta 0), reused 1 (delta 0), pack-reused 75
Unpacking objects: 100% (79/79), done.


构建镜像并打好 tag：

In [5]:
!docker build ecs-demo-php-simple-app -t ecs-demo-php-simple-app

Sending build context to Docker daemon  380.9kB
Step 1/10 : FROM public.ecr.aws/amazonlinux/amazonlinux:2
2: Pulling from amazonlinux/amazonlinux

Digest: sha256:69a7e4782be3dc25f610582d6ad7a88d1db7db08f27248153efe726ea6077bbf[1A
Status: Downloaded newer image for public.ecr.aws/amazonlinux/amazonlinux:2
 ---> d7b15f736b7e
Step 2/10 : RUN yum install -y     curl     httpd     php  && ln -s /usr/sbin/httpd /usr/sbin/apache2
 ---> Running in f6e169a59aa5
Loaded plugins: ovl, priorities
Package curl-7.76.1-7.amzn2.0.2.x86_64 already installed and latest version
Resolving Dependencies
--> Running transaction check
---> Package httpd.x86_64 0:2.4.51-1.amzn2 will be installed
--> Processing Dependency: httpd-tools = 2.4.51-1.amzn2 for package: httpd-2.4.51-1.amzn2.x86_64
--> Processing Dependency: httpd-filesystem = 2.4.51-1.amzn2 for package: httpd-2.4.51-1.amzn2.x86_64
--> Processing Dependency: systemd-units for package: httpd-2.4.51-1.amzn2.x86_64
--> Processing Dependency: systemd-units

In [6]:
!docker tag ecs-demo-php-simple-app {aws_account_id}.dkr.ecr.{aws_region}.amazonaws.com/{repository_name}

可以通过以下命令确认容器镜像已经构建成功：

In [7]:
!docker images

REPOSITORY                                                             TAG       IMAGE ID       CREATED              SIZE
364198545638.dkr.ecr.us-east-1.amazonaws.com/ecs-demo-php-simple-app   latest    b7100a00e52a   About a minute ago   529MB
ecs-demo-php-simple-app                                                latest    b7100a00e52a   About a minute ago   529MB
public.ecr.aws/amazonlinux/amazonlinux                                 2         d7b15f736b7e   7 days ago           164MB


接下来，登录到 ECR 并将镜像推送到仓库：

In [8]:
!aws ecr get-login-password | docker login --username AWS --password-stdin {aws_account_id}.dkr.ecr.{aws_region}.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [9]:
!docker push {aws_account_id}.dkr.ecr.{aws_region}.amazonaws.com/{repository_name}

Using default tag: latest
The push refers to repository [364198545638.dkr.ecr.us-east-1.amazonaws.com/ecs-demo-php-simple-app]

5583b9e3: Preparing 
820811d7: Preparing 
a763db23: Preparing 
a763db23: Pushed   367.8MB/364.5MBlatest: digest: sha256:2413c6094d69e7960f4b20df153bf4ed9e444c66418e41b0a980c085ef9e73a5 size: 1160


### 创建 ECS 任务定义

接下来，我们在 ECS 服务中创建与镜像相对应的 task definition：

In [11]:
ecs.register_task_definition(
    family="sample-app",
    volumes=[
        {
            "name": "my-vol",
            "host": {}
        }
    ],
    networkMode="awsvpc",
    executionRoleArn="arn:aws:iam::{}:role/ecsTaskExecutionRole".format(aws_account_id),
    containerDefinitions=[
        {
            "name": "simple-app",
            "image": "{}.dkr.ecr.{}.amazonaws.com/{}:latest".format(aws_account_id, aws_region, repository_name),
            "portMappings": [
                {
                    "containerPort": 80,
                    "hostPort": 80,
                    "protocol": "tcp"
                }
            ],
            "mountPoints": [
                {
                    "sourceVolume": "my-vol",
                    "containerPath": "/var/www/my-vol"
                }
            ],
            "entryPoint": [
                "/usr/sbin/apache2",
                "-D",
                "FOREGROUND"
            ],
            "essential": True
        },
        {
            "name": "busybox",
            "image": "busybox",
            "cpu": 10,
            "memory": 500,
            "volumesFrom": [
            {
              "sourceContainer": "simple-app"
            }
            ],
            "entryPoint": [
                "sh",
                "-c"
            ],
            "command": [
                "/bin/sh -c \"while true; do /bin/date > /var/www/my-vol/date; sleep 1; done\""
            ],
            "essential": False
        }
    ],
    requiresCompatibilities=["FARGATE"],
    memory="512",
    cpu="256"
)

{'taskDefinition': {'taskDefinitionArn': 'arn:aws:ecs:us-east-1:364198545638:task-definition/sample-app:2',
  'containerDefinitions': [{'name': 'simple-app',
    'image': '364198545638.dkr.ecr.us-east-1.amazonaws.com/ecs-demo-php-simple-app:latest',
    'cpu': 0,
    'portMappings': [{'containerPort': 80, 'hostPort': 80, 'protocol': 'tcp'}],
    'essential': True,
    'entryPoint': ['/usr/sbin/apache2', '-D', 'FOREGROUND'],
    'environment': [],
    'mountPoints': [{'sourceVolume': 'my-vol',
      'containerPath': '/var/www/my-vol'}],
    'volumesFrom': []},
   {'name': 'busybox',
    'image': 'busybox',
    'cpu': 10,
    'memory': 500,
    'portMappings': [],
    'essential': False,
    'entryPoint': ['sh', '-c'],
    'command': ['/bin/sh -c "while true; do /bin/date > /var/www/my-vol/date; sleep 1; done"'],
    'environment': [],
    'mountPoints': [],
    'volumesFrom': [{'sourceContainer': 'simple-app'}]}],
  'family': 'sample-app',
  'executionRoleArn': 'arn:aws:iam::36419854563

### 运行示例程序

再接下来，复制上面的'taskDefinitionArn'，再到 CloudFormation Output 中找到公有子网的 id 和 http 安全组 id：

In [12]:
task_definition = "arn:aws:ecs:us-east-1:364198545638:task-definition/sample-app:1"
subnet1 = "subnet-0528c12eb7ea9d563"
subnet2 = "subnet-01c6254baebb0e50c"
subnet3 = "subnet-09d7d3e9ebf23ba05"
sg = "sg-08bca067008951a4d"

完成上述步骤之后，就可以再 Fargate 中创建容器来运行示例程序了：

In [13]:
ecs.run_task(
    cluster='algo-trading-workshop',
    enableExecuteCommand=False,
    group='family:algo-trading-workshop',
    launchType='FARGATE',
    count=2,
    networkConfiguration={
                'awsvpcConfiguration': {
                    'subnets': [subnet1, subnet2, subnet3],
                    'securityGroups': [sg],
                    'assignPublicIp': 'ENABLED'
                }
            },
    taskDefinition=task_definition
)

{'tasks': [{'attachments': [{'id': 'd46f31f6-47e1-4596-bef3-5d8e70e9df95',
     'type': 'ElasticNetworkInterface',
     'status': 'PRECREATED',
     'details': [{'name': 'subnetId', 'value': 'subnet-01c6254baebb0e50c'}]}],
   'attributes': [{'name': 'ecs.cpu-architecture', 'value': 'x86_64'}],
   'availabilityZone': 'us-east-1b',
   'clusterArn': 'arn:aws:ecs:us-east-1:364198545638:cluster/algo-trading-workshop',
   'containers': [{'containerArn': 'arn:aws:ecs:us-east-1:364198545638:container/algo-trading-workshop/e3cf22dcc6c6496ca793d2a7a4d66621/3fe2809c-5a1c-4f6e-ad5c-14ce88231fef',
     'taskArn': 'arn:aws:ecs:us-east-1:364198545638:task/algo-trading-workshop/e3cf22dcc6c6496ca793d2a7a4d66621',
     'name': 'simple-app',
     'image': '364198545638.dkr.ecr.us-east-1.amazonaws.com/ecs-demo-php-simple-app:latest',
     'lastStatus': 'PENDING',
     'networkInterfaces': [],
     'cpu': '0'},
    {'containerArn': 'arn:aws:ecs:us-east-1:364198545638:container/algo-trading-workshop/e3cf22d

最后再找到这个 task 相应的 公网ip，就可以访问这个示例 PHP 服务了。确认服务能够正常运行就可以把停止 task 了。